In [53]:
from label_studio_sdk import Client

# Initialize the client
ls = Client(url='http://10.0.101.10:8080', api_key='9128fab0ec30016774d8ae41fb84f541a7955817')

# Get the project
project = ls.get_project(23)

# Export tasks
tasks = project.export_tasks(export_type='JSON', download_all_tasks=True)

In [13]:
import json
import os

# === CONFIGURACIÓN ===

json_path = "/workspace/RGB_T_SAR/export_v_II/lucas_rgb_poligonos.json"  # Tu archivo de Label Studio
output_dir = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_lucas_rgb/"  # Carpeta donde guardarás los JSON individuales

os.makedirs(output_dir, exist_ok=True)

# === Cargar JSON principal ===
with open(json_path, "r") as f:
    all_tasks = json.load(f)

# === Guardar cada task en un archivo separado ===
for i, task in enumerate(all_tasks):
    output_path = os.path.join(output_dir, f"task_{i+1:04}.json")
    with open(output_path, "w") as out_f:
        json.dump(task, out_f, indent=2)

print(f"✅ Se dividieron {len(all_tasks)} tareas en archivos individuales en:\n{output_dir}")


✅ Se dividieron 20 tareas en archivos individuales en:
/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_lucas_rgb/


In [14]:
import json
import os
import numpy as np
from PIL import Image, ImageDraw

# === CONFIGURACIÓN ===

input_json_folder = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_lucas_rgb/"  # Carpeta con tus JSON
output_mask_folder = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_png_lucas_rgb/"
os.makedirs(output_mask_folder, exist_ok=True)

# === Mapa FIJO de clases: nombre (Label Studio) -> ID entero ===
# OJO: estas cadenas deben coincidir EXACTAMENTE con 'polygonlabels' en los JSON
CLASS_TO_ID = {
    "First-Responder": 1,
    "Civilian": 2,
    "Vegetation": 3,
    "Road": 4,
    "Dirt-Road": 5,
    "Building": 6,
    "Sky": 7,
    "Civilian-Car": 8,
    "Responder-Vehicle": 9,
    "Debris": 10,
    "Command-Post": 11,
}
# 0 queda como fondo / unlabeled

# === Procesar todos los archivos JSON ===
json_files = [f for f in os.listdir(input_json_folder) if f.endswith('.json')]

for json_file in json_files:
    json_path = os.path.join(input_json_folder, json_file)

    with open(json_path, 'r') as f:
        data = json.load(f)

    # Soporta tanto lista de tareas como un solo dict
    task = data[0] if isinstance(data, list) else data
    annotations = task['annotations'][0]['result']

    if len(annotations) == 0:
        print(f"⚠ {json_file} no tiene anotaciones, se salta.")
        continue

    # Detectar altura y anchura a partir de la primera anotación
    first_ann = annotations[0]
    height = first_ann.get('original_height')
    width = first_ann.get('original_width')

    if height is None or width is None:
        raise ValueError(f"No se encontraron 'original_height'/'original_width' en {json_file}")

    # Creamos la máscara como imagen PIL para poder dibujar polígonos
    # Fondo = 0 (unlabeled)
    mask_img = Image.new(mode="L", size=(width, height), color=0)
    draw = ImageDraw.Draw(mask_img)

    for ann in annotations:
        if 'value' not in ann:
            print(f"   → Saltando anotación sin 'value' en {json_file}")
            continue
        if 'polygonlabels' not in ann['value']:
            print(f"   → Saltando anotación sin 'polygonlabels' en {json_file}")
            continue
        if 'points' not in ann['value']:
            print(f"   → Saltando anotación sin 'points' en {json_file}")
            continue

        label = ann['value']['polygonlabels'][0]

        # Obtener ID de clase a partir del mapa fijo
        class_id = CLASS_TO_ID.get(label, 0)  # si no está en el diccionario → 0 (unlabeled)

        if label not in CLASS_TO_ID:
            print(f"⚠ Etiqueta '{label}' no está en CLASS_TO_ID, se marca como 0 (unlabeled) en {json_file}")

        points = ann['value']['points']

        # Convertir puntos normalizados (0–100) a píxeles
        poly_pts = [
            (p[0] * width / 100.0, p[1] * height / 100.0)
            for p in points
        ]

        # Dibujar el polígono lleno con el valor de la clase
        draw.polygon(poly_pts, outline=class_id, fill=class_id)

    # Convertir la máscara PIL a numpy (por si luego quieres procesarla)
    mask = np.array(mask_img, dtype=np.uint8)

    # Nombre de salida (basado en el nombre del JSON)
    base_name = os.path.splitext(json_file)[0]
    output_path = os.path.join(output_mask_folder, f"{base_name}.png")
    Image.fromarray(mask).save(output_path)
    print(f"✅ Máscara (polígonos) guardada: {output_path}")

print("\n✅ Todos los JSON de polígonos han sido convertidos a PNG con IDs de clase fijos.")




✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_png_lucas_rgb/task_0001.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_png_lucas_rgb/task_0002.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_png_lucas_rgb/task_0003.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_png_lucas_rgb/task_0004.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_png_lucas_rgb/task_0005.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_png_lucas_rgb/task_0006.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_o

In [15]:
import json
import os
import numpy as np
from PIL import Image, ImageDraw   # 👈 ojo: ahora necesitamos ImageDraw

# === CONFIGURACIÓN ===
input_json_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_lucas_rgb/"
output_color_mask_folder = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_color_lucas_rgb/"
os.makedirs(output_color_mask_folder, exist_ok=True)

# === Diccionario de colores específicos por clase (RGB) ===
class_colors = {
    "First-Responder": (255, 132, 0),
    "Civilian": (224, 67, 216),
    "Vegetation": (65, 117, 5),
    "Road": (155, 155, 155),
    "Dirt-Road": (255, 221, 104),
    "Building": (180, 117, 31),
    "Sky": (22, 152, 207),
    "Civilian-Car": (64, 0, 128),
    "Responder-Vehicle": (192, 64, 0),
    "Debris": (60, 37, 0),
    "Command-Post": (192, 128, 128)
}

# === Procesar todos los archivos JSON ===
json_files = [f for f in os.listdir(input_json_folder) if f.endswith('.json')]

for json_file in json_files:
    json_path = os.path.join(input_json_folder, json_file)

    with open(json_path, 'r') as f:
        data = json.load(f)

    # Soportar tanto lista de tareas como un solo dict
    task = data[0] if isinstance(data, list) else data
    annotations = task['annotations'][0]['result']

    # Mapas de clases
    label_map = {}
    label_value = 1

    # === Detectar altura y anchura a partir de la primera anotación ===
    first_ann = annotations[0]
    height = first_ann.get('original_height')
    width = first_ann.get('original_width')

    if height is None or width is None:
        raise ValueError(f"No se encontraron 'original_height'/'original_width' en {json_file}")

    # === Máscara indexada (clase por píxel) como imagen PIL en escala de grises ===
    mask_img = Image.new(mode="L", size=(width, height), color=0)
    draw = ImageDraw.Draw(mask_img)

    for ann in annotations:
        if 'value' not in ann:
            print(f"Saltando anotación sin 'value' en {json_file}")
            continue
        if 'polygonlabels' not in ann['value']:
            print(f"Saltando anotación sin 'polygonlabels' en {json_file}")
            continue
        if 'points' not in ann['value']:
            print(f"Saltando anotación sin 'points' en {json_file}")
            continue

        label = ann['value']['polygonlabels'][0]

        # Asignar un índice numérico a cada clase
        if label not in label_map:
            label_map[label] = label_value
            label_value += 1

        class_id = label_map[label]
        points = ann['value']['points']

        # === Convertir puntos normalizados (0–100) a píxeles ===
        # Si tus puntos ya están en píxeles, reemplaza este bloque por:
        # poly_pts = [(p[0], p[1]) for p in points]
        poly_pts = [
            (p[0] * width / 100.0, p[1] * height / 100.0)
            for p in points
        ]

        # Dibujar el polígono con el valor de la clase
        draw.polygon(poly_pts, outline=class_id, fill=class_id)

    # Convertir la máscara PIL a numpy (índices de clase)
    mask = np.array(mask_img, dtype=np.uint8)

    # === Crear y guardar máscara coloreada usando los colores fijos ===
    mask_rgb = np.zeros((height, width, 3), dtype=np.uint8)

    for label_name, label_idx in label_map.items():
        # Color específico si está en el diccionario; blanco si no
        if label_name in class_colors:
            color = np.array(class_colors[label_name], dtype=np.uint8)
        else:
            color = np.array([255, 255, 255], dtype=np.uint8)

        mask_rgb[mask == label_idx] = color

    base_name = os.path.splitext(json_file)[0]
    color_output_path = os.path.join(output_color_mask_folder, f"{base_name}.png")
    Image.fromarray(mask_rgb).save(color_output_path)
    print(f"✅ Máscara coloreada (polígonos) guardada: {color_output_path}")

print("\n✅ Todos los JSON de polígonos han sido convertidos a máscaras coloreadas PNG.")


✅ Máscara coloreada (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_color_lucas_rgb/task_0001.png
✅ Máscara coloreada (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_color_lucas_rgb/task_0002.png
✅ Máscara coloreada (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_color_lucas_rgb/task_0003.png
✅ Máscara coloreada (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_color_lucas_rgb/task_0004.png
✅ Máscara coloreada (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_color_lucas_rgb/task_0005.png
✅ Máscara coloreada (polígonos) guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/poligonos/json_to_color_lucas_rgb/task_0

In [58]:
import os
import json

# === CONFIGURACIÓN ===
# Carpeta donde tienes los JSON originales
INPUT_DIR = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_rgb_sam2"

# Carpeta donde quieres guardar los JSON limpios
OUTPUT_DIR = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_rgb_sam2_limpio"

# Prefijo actual de la ruta de la imagen en "data.image"
# Tal como aparece en tus JSON:
# "data": {
#     "image": "/data/local-files/?d=proyecto_oct_2025_sam2/dataset/dataset_2025/rgb/0001.png"
# }
OLD_IMAGE_PREFIX = "/data/local-files/?d=proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/rgb_images/"


# Nuevo prefijo que quieres usar (MODIFICA ESTO A TU GUSTO)
NEW_IMAGE_PREFIX = "/data/local-files/?d=proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/tir_images/"


def clean_task(task: dict) -> dict:
    """Elimina 'id' en el nivel superior y dentro de 'annotations',
    y actualiza la ruta de data.image si corresponde."""
    # Quitar id del nivel superior
    task.pop("id", None)

    # Modificar ruta de la imagen si existe
    try:
        image_path = task["data"]["image"]
        if isinstance(image_path, str) and OLD_IMAGE_PREFIX and image_path.startswith(OLD_IMAGE_PREFIX):
            # Reemplaza solo el prefijo
            task["data"]["image"] = image_path.replace(OLD_IMAGE_PREFIX, NEW_IMAGE_PREFIX, 1)
    except (KeyError, TypeError):
        # Si no hay data o image, lo ignoramos
        pass

    # Limpiar annotations
    annotations = task.get("annotations", [])
    if isinstance(annotations, list):
        for ann in annotations:
            if isinstance(ann, dict):
                ann.pop("id", None)
                ann.pop("unique_id", None)

    return task


def process_file(input_path: str, output_path: str):
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # El export de Label Studio a veces es una lista de tareas, o una sola tarea
    if isinstance(data, list):
        cleaned = [clean_task(task) for task in data if isinstance(task, dict)]
    elif isinstance(data, dict):
        cleaned = clean_task(data)
    else:
        print(f"Formato no esperado en {input_path}, se omite.")
        return

    # Guardar JSON limpio
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(cleaned, f, ensure_ascii=False, indent=2)

    print(f"Procesado: {os.path.basename(input_path)}  ->  {os.path.basename(output_path)}")


def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    for fname in os.listdir(INPUT_DIR):
        if not fname.lower().endswith(".json"):
            continue

        in_path = os.path.join(INPUT_DIR, fname)
        out_path = os.path.join(OUTPUT_DIR, fname)

        process_file(in_path, out_path)


if __name__ == "__main__":
    main()




Procesado: task_0001.json  ->  task_0001.json
Procesado: task_0002.json  ->  task_0002.json
Procesado: task_0003.json  ->  task_0003.json
Procesado: task_0004.json  ->  task_0004.json
Procesado: task_0005.json  ->  task_0005.json
Procesado: task_0006.json  ->  task_0006.json
Procesado: task_0007.json  ->  task_0007.json
Procesado: task_0008.json  ->  task_0008.json
Procesado: task_0009.json  ->  task_0009.json
Procesado: task_0010.json  ->  task_0010.json
Procesado: task_0011.json  ->  task_0011.json
Procesado: task_0012.json  ->  task_0012.json
Procesado: task_0013.json  ->  task_0013.json
Procesado: task_0014.json  ->  task_0014.json
Procesado: task_0015.json  ->  task_0015.json
Procesado: task_0016.json  ->  task_0016.json
Procesado: task_0017.json  ->  task_0017.json
Procesado: task_0018.json  ->  task_0018.json
Procesado: task_0019.json  ->  task_0019.json
Procesado: task_0020.json  ->  task_0020.json
Procesado: task_0021.json  ->  task_0021.json
Procesado: task_0022.json  ->  tas

In [39]:
# Tomar las imagenes de las etiquetas json del dataset mfnet

In [55]:
import json
import os
import numpy as np
from PIL import Image, ImageDraw

# === CONFIGURACIÓN ===

input_json_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_tir_poligono/"  # Carpeta con tus JSON
output_mask_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_png_poligono_tir/"
os.makedirs(output_mask_folder, exist_ok=True)

# === Mapa de clases FIJO ===
# Asegúrate de que estas cadenas coincidan EXACTAMENTE con los 'polygonlabels' del JSON
CLASS_MAP = {
    "car": 1,
    "person": 2,
    "bike": 3,
    "curve": 4,
    "car_stop": 5,
    "guardrail ": 6,
    "color_cone": 7,
    "bump": 8,
}
# Cualquier etiqueta no encontrada en CLASS_MAP será "unlabeled" = 0

# === Procesar todos los archivos JSON ===
json_files = [f for f in os.listdir(input_json_folder) if f.endswith('.json')]

for json_file in json_files:
    json_path = os.path.join(input_json_folder, json_file)

    with open(json_path, 'r') as f:
        data = json.load(f)

    # Soporta tanto lista de tareas como un solo dict
    task = data[0] if isinstance(data, list) else data

    # Si no hay anotaciones, omitir este JSON
    if 'annotations' not in task or not task['annotations']:
        print(f"⚠ {json_file}: sin anotaciones, se omite.")
        continue

    # Tomar la primera anotación
    ann0 = task['annotations'][0]

    # Si no tiene 'result' o está vacío, también omitir
    if 'result' not in ann0 or not ann0['result']:
        print(f"⚠ {json_file}: anotación sin 'result', se omite.")
        continue

    annotations = ann0['result']


    # Detectar altura y anchura a partir de la primera anotación
    first_ann = annotations[0]
    height = first_ann.get('original_height')
    width = first_ann.get('original_width')

    if height is None or width is None:
        raise ValueError(f"No se encontraron 'original_height'/'original_width' en {json_file}")

    # Creamos la máscara como imagen PIL para poder dibujar polígonos
    # Fondo = 0 (unlabeled)
    mask_img = Image.new(mode="L", size=(width, height), color=0)
    draw = ImageDraw.Draw(mask_img)

    for ann in annotations:
        if 'value' not in ann:
            print(f"Saltando anotación sin 'value' en {json_file}")
            continue
        if 'polygonlabels' not in ann['value']:
            print(f"Saltando anotación sin 'polygonlabels' en {json_file}")
            continue
        if 'points' not in ann['value']:
            print(f"Saltando anotación sin 'points' en {json_file}")
            continue

        label = ann['value']['polygonlabels'][0]

        # Obtener ID de clase a partir del mapa fijo
        # Si la etiqueta no está en CLASS_MAP → class_id = 0 (unlabeled)
        class_id = CLASS_MAP.get(label, 0)
        if class_id == 0 and label not in CLASS_MAP:
            print(f"⚠️ Etiqueta '{label}' no está en CLASS_MAP; se marcará como 0 (unlabeled) en {json_file}")

        points = ann['value']['points']

        # === Convertir puntos normalizados (0-100) a píxeles ===
        poly_pts = [
            (p[0] * width / 100.0, p[1] * height / 100.0)
            for p in points
        ]

        # Dibujar el polígono lleno con el valor de la clase
        draw.polygon(poly_pts, outline=class_id, fill=class_id)

    # Convertir la máscara PIL a numpy (por si luego quieres procesarla)
    mask = np.array(mask_img, dtype=np.uint8)

    # Nombre de salida (basado en el nombre del JSON)
    base_name = os.path.splitext(json_file)[0]
    output_path = os.path.join(output_mask_folder, f"{base_name}.png")
    Image.fromarray(mask).save(output_path)
    print(f"✅ Máscara (polígonos) guardada: {output_path}")

print("\n✅ Todos los JSON de polígonos han sido convertidos a PNG con el esquema de clases fijo.")


✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_png_poligono_tir/task_0001.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_png_poligono_tir/task_0002.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_png_poligono_tir/task_0003.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_png_poligono_tir/task_0004.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_png_poligono_tir/task_0005.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_png_poligono_tir/task_0006.png
✅ Máscara (polígonos) guardada: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/

In [31]:
import numpy as np
from PIL import Image

def compute_iou_per_class(gt_mask, pred_mask, num_classes=9, ignore_index=0):
    """
    Calcula IoU por clase y mIoU.
    gt_mask, pred_mask: arrays 2D con IDs de clase
    num_classes: número total de clases (aquí 9: clases 0..8)
    ignore_index: clase a ignorar en el cálculo de mIoU (0 = unlabeled)
    """
    iou_per_class = {}

    for c in range(num_classes):
        gt_c = (gt_mask == c)
        pred_c = (pred_mask == c)

        intersection = np.logical_and(gt_c, pred_c).sum()
        union = np.logical_or(gt_c, pred_c).sum()

        if union == 0:
            iou = np.nan  # no hay píxeles de esta clase en ninguna máscara
        else:
            iou = intersection / union

        iou_per_class[c] = iou

    # Calcular mIoU excluyendo clase ignorada y clases sin píxeles
    valid_ious = [
        v for k, v in iou_per_class.items()
        if (ignore_index is None or k != ignore_index) and not np.isnan(v)
    ]

    miou = np.mean(valid_ious) if len(valid_ious) > 0 else np.nan

    return iou_per_class, miou


In [32]:
# RUTAS A TUS DOS PNG (cámbialas según corresponda)
gt_path = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/01512D (1).png"      # Ground truth
pred_path = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/task_0002.png"  # Máscara a comparar

# Cargar imágenes como arrays
gt = np.array(Image.open(gt_path))
pred = np.array(Image.open(pred_path))

print("GT shape:", gt.shape)
print("Pred shape:", pred.shape)

# Validación
assert gt.shape == pred.shape, "Las máscaras deben tener el mismo tamaño"


GT shape: (480, 640)
Pred shape: (480, 640)


In [33]:
# Número de clases: 0..8 (9 clases en total)
NUM_CLASSES = 9  

# Clase a ignorar en mIoU (0 = unlabeled)
IGNORE_CLASS = 0  

iou_per_class, miou = compute_iou_per_class(
    gt, pred, num_classes=NUM_CLASSES, ignore_index=IGNORE_CLASS
)

print("=== IoU por clase ===")
for cls_id, iou in iou_per_class.items():
    if np.isnan(iou):
        print(f"Clase {cls_id}: NaN (sin píxeles en GT y Pred)")
    else:
        print(f"Clase {cls_id}: {iou:.4f}")

print(f"\n✅ mIoU (sin clase {IGNORE_CLASS}): {miou:.4f}")


=== IoU por clase ===
Clase 0: 0.9919
Clase 1: 0.8093
Clase 2: 0.8693
Clase 3: NaN (sin píxeles en GT y Pred)
Clase 4: NaN (sin píxeles en GT y Pred)
Clase 5: NaN (sin píxeles en GT y Pred)
Clase 6: NaN (sin píxeles en GT y Pred)
Clase 7: NaN (sin píxeles en GT y Pred)
Clase 8: NaN (sin píxeles en GT y Pred)

✅ mIoU (sin clase 0): 0.8393


In [60]:
import os
import numpy as np
from PIL import Image

def compute_iou_per_class(gt_mask, pred_mask, num_classes=9, ignore_index=0):
    iou_per_class = {}

    for c in range(num_classes):
        gt_c = (gt_mask == c)
        pred_c = (pred_mask == c)

        intersection = np.logical_and(gt_c, pred_c).sum()
        union = np.logical_or(gt_c, pred_c).sum()

        if union == 0:
            iou = np.nan
        else:
            iou = intersection / union

        iou_per_class[c] = iou

    valid_ious = [
        v for k, v in iou_per_class.items()
        if (ignore_index is None or k != ignore_index) and not np.isnan(v)
    ]

    miou = np.mean(valid_ious) if len(valid_ious) > 0 else np.nan
    return iou_per_class, miou


# ============================================================
#  PROCESAR TODA UNA CARPETA
# ============================================================

GT_FOLDER = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/ground_truth/"
PRED_FOLDER = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_png_mixto_tir/"
NUM_CLASSES = 9
IGNORE_INDEX = None  # 0 = unlabeled

gt_files = sorted([f for f in os.listdir(GT_FOLDER) if f.endswith(".png")])

global_ious = []
print("\n======================== RESULTADOS POR IMAGEN ========================\n")

for fname in gt_files:
    gt_path = os.path.join(GT_FOLDER, fname)
    pred_path = os.path.join(PRED_FOLDER, fname)

    if not os.path.exists(pred_path):
        print(f"⚠️  No existe predicción para: {fname}, se omite.")
        continue

    gt_mask = np.array(Image.open(gt_path))
    pred_mask = np.array(Image.open(pred_path))

    # Verificar tamaños
    if gt_mask.shape != pred_mask.shape:
        print(f"⚠️ Tamaño diferente entre GT y Pred para {fname}, se omite.")
        continue

    iou_c, miou = compute_iou_per_class(gt_mask, pred_mask,
                                        num_classes=NUM_CLASSES,
                                        ignore_index=IGNORE_INDEX)

    global_ious.append(miou)

    print(f"📌 {fname}: mIoU = {miou:.4f}")
    # Si quieres ver IoU por clase, activa esto:
    # print("   IoU por clase:", iou_c)

print("\n======================== RESULTADO GLOBAL ========================\n")

if len(global_ious) > 0:
    final_miou = np.mean(global_ious)
    print(f"🏁 mIoU PROMEDIO DE TODA LA CARPETA = {final_miou:.4f}")
else:
    print("❌ No se calcularon IoUs (ninguna imagen válida).")



======================== RESULTADOS POR IMAGEN ========================

📌 task_0001.png: mIoU = 0.9553
📌 task_0002.png: mIoU = 0.8976
📌 task_0003.png: mIoU = 0.5285
📌 task_0004.png: mIoU = 0.5170
📌 task_0005.png: mIoU = 0.7433
📌 task_0006.png: mIoU = 0.7261
📌 task_0007.png: mIoU = 0.6825
📌 task_0008.png: mIoU = 0.4744
📌 task_0009.png: mIoU = 0.9478
📌 task_0010.png: mIoU = 0.9396
📌 task_0011.png: mIoU = 0.8526
📌 task_0012.png: mIoU = 0.7700
📌 task_0013.png: mIoU = 0.8239
📌 task_0014.png: mIoU = 0.9480
📌 task_0015.png: mIoU = 0.6215
📌 task_0016.png: mIoU = 0.8472
📌 task_0017.png: mIoU = 0.6880
📌 task_0018.png: mIoU = 0.8233
📌 task_0019.png: mIoU = 0.5074
📌 task_0020.png: mIoU = 0.5698
📌 task_0021.png: mIoU = 0.8198
📌 task_0022.png: mIoU = 0.7862
📌 task_0023.png: mIoU = 0.8734
📌 task_0024.png: mIoU = 0.7621
📌 task_0025.png: mIoU = 0.7005
📌 task_0026.png: mIoU = 0.7390
📌 task_0027.png: mIoU = 0.5591
📌 task_0028.png: mIoU = 0.7873
📌 task_0029.png: mIoU = 0.7985
📌 task_0030.png: mIoU = 0.7

In [64]:
import json
import os
import numpy as np
from PIL import Image, ImageDraw

# === CONFIGURACIÓN ===
input_json_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_rgb_poligono"
output_color_mask_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_rgb_polig/"
os.makedirs(output_color_mask_folder, exist_ok=True)

# === Diccionario de colores específicos por clase (RGB) ===
class_colors = {
    "unlabelled": (0, 0, 0),
    "car":        (64, 0, 128),
    "person":     (64, 64, 0),
    "bike":       (0, 128, 192),
    "curve":      (0, 0, 192),
    "car_stop":   (128, 128, 0),
    "guardrail ":  (64, 64, 128),
    "color_cone": (192, 128, 128),
    "bump":       (192, 64, 0)
}

json_files = [f for f in os.listdir(input_json_folder) if f.endswith(".json")]

skipped = 0
processed = 0

for json_file in json_files:
    json_path = os.path.join(input_json_folder, json_file)

    with open(json_path, "r") as f:
        data = json.load(f)

    task = data[0] if isinstance(data, list) else data

    # ✅ Robust checks to avoid IndexError
    anns = task.get("annotations", [])
    if not isinstance(anns, list) or len(anns) == 0:
        print(f"⚠️ Skipping (no annotations list): {json_file}")
        skipped += 1
        continue

    first_ann_block = anns[0]
    if not isinstance(first_ann_block, dict):
        print(f"⚠️ Skipping (annotations[0] not a dict): {json_file}")
        skipped += 1
        continue

    annotations = first_ann_block.get("result", [])
    if not isinstance(annotations, list) or len(annotations) == 0:
        print(f"⚠️ Skipping (no result polygons): {json_file}")
        skipped += 1
        continue

    # === Detectar altura y anchura a partir de la primera anotación ===
    height = annotations[0].get("original_height")
    width  = annotations[0].get("original_width")
    if height is None or width is None:
        print(f"⚠️ Skipping (missing original_height/width): {json_file}")
        skipped += 1
        continue

    mask_img = Image.new(mode="L", size=(width, height), color=0)
    draw = ImageDraw.Draw(mask_img)

    label_map = {}
    label_value = 1

    for ann in annotations:
        v = ann.get("value", {})
        poly_labels = v.get("polygonlabels", None)
        points = v.get("points", None)

        if not poly_labels or not points:
            continue

        label = poly_labels[0]

        if label not in label_map:
            label_map[label] = label_value
            label_value += 1

        class_id = label_map[label]

        poly_pts = [(p[0] * width / 100.0, p[1] * height / 100.0) for p in points]
        draw.polygon(poly_pts, outline=class_id, fill=class_id)

    mask = np.array(mask_img, dtype=np.uint8)

    mask_rgb = np.zeros((height, width, 3), dtype=np.uint8)
    for label_name, label_idx in label_map.items():
        color = np.array(class_colors.get(label_name, (255, 255, 255)), dtype=np.uint8)
        mask_rgb[mask == label_idx] = color

    base_name = os.path.splitext(json_file)[0]
    color_output_path = os.path.join(output_color_mask_folder, f"{base_name}.png")
    Image.fromarray(mask_rgb).save(color_output_path)

    print(f"✅ Saved: {color_output_path}")
    processed += 1

print(f"\nDone. Processed: {processed} | Skipped: {skipped}")


✅ Saved: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_rgb_polig/task_0001.png
✅ Saved: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_rgb_polig/task_0002.png
✅ Saved: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_rgb_polig/task_0003.png
✅ Saved: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_rgb_polig/task_0004.png
⚠️ Skipping (no annotations list): task_0005.json
✅ Saved: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_rgb_polig/task_0006.png
✅ Saved: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_rgb_polig/task_0007.png
✅ Saved: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_rgb_polig/task_0008.png
✅ Saved: /workspace/RGB_T_SAR/proyecto_o

In [67]:
import json
import os
import numpy as np
from PIL import Image, ImageDraw
from label_studio_sdk.converter.brush import decode_rle

# === CONFIGURACIÓN ===
input_json_folder = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_tir_sam2/"
output_color_mask_folder = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_sam2_tir/"
os.makedirs(output_color_mask_folder, exist_ok=True)

# === Diccionario de colores específicos por clase (RGB) ===
class_colors = {
    "unlabelled": (0, 0, 0),
    "car":        (64, 0, 128),
    "person":     (64, 64, 0),
    "bike":       (0, 128, 192),
    "curve":      (0, 0, 192),
    "car_stop":   (128, 128, 0),
    "guardrail ":  (64, 64, 128),
    "color_cone": (192, 128, 128),
    "bump":       (192, 64, 0)
}

# === Procesar todos los archivos JSON ===
json_files = [f for f in os.listdir(input_json_folder) if f.endswith(".json")]

for json_file in json_files:
    json_path = os.path.join(input_json_folder, json_file)

    with open(json_path, "r") as f:
        data = json.load(f)

    task = data[0] if isinstance(data, list) else data

    # ✅ Skip if no annotations
    anns = task.get("annotations", [])
    if not isinstance(anns, list) or len(anns) == 0:
        print(f"⚠️ Skipping (no annotations): {json_file}")
        continue

    annotations = anns[0].get("result", [])
    if not isinstance(annotations, list) or len(annotations) == 0:
        print(f"⚠️ Skipping (empty result): {json_file}")
        continue

    # === Detectar altura y anchura (robusto) ===
    height = None
    width = None

    # 1) intentar desde original_height/original_width
    for ann in annotations:
        if ann.get("original_height") and ann.get("original_width"):
            height = int(ann["original_height"])
            width  = int(ann["original_width"])
            break

    # 2) si no, intentar desde rle dict {size: [h,w]}
    if height is None or width is None:
        for ann in annotations:
            v = ann.get("value", {})
            rle = v.get("rle", None)
            if isinstance(rle, dict) and "size" in rle:
                height, width = rle["size"]
                height, width = int(height), int(width)
                break

    if height is None or width is None:
        print(f"⚠️ Skipping (cannot infer H/W): {json_file}")
        continue

    # === Máscara indexada por clase ===
    mask = np.zeros((height, width), dtype=np.uint8)

    # Polígonos se dibujan con PIL (más rápido y consistente)
    poly_img = Image.new("L", (width, height), 0)
    poly_draw = ImageDraw.Draw(poly_img)

    # Mapas de clases (por archivo) como tu estilo
    label_map = {}
    label_value = 1

    # ===== 1) Procesar POLÍGONOS =====
    for ann in annotations:
        v = ann.get("value", {})
        if "polygonlabels" not in v or "points" not in v:
            continue

        labels = v.get("polygonlabels", [])
        points = v.get("points", [])
        if not labels or not points:
            continue

        label = labels[0]
        if label not in label_map:
            label_map[label] = label_value
            label_value += 1

        class_id = label_map[label]

        # points normalizados (0-100) -> píxeles
        poly_pts = [(p[0] * width / 100.0, p[1] * height / 100.0) for p in points]
        poly_draw.polygon(poly_pts, outline=class_id, fill=class_id)

    # Volcar polígonos a mask
    poly_mask = np.array(poly_img, dtype=np.uint8)
    mask[poly_mask > 0] = poly_mask[poly_mask > 0]

    # ===== 2) Procesar BRUSH (RLE) =====
    for ann in annotations:
        v = ann.get("value", {})
        if "brushlabels" not in v or "rle" not in v:
            continue

        labels = v.get("brushlabels", [])
        if not labels:
            continue

        label = labels[0]
        if label not in label_map:
            label_map[label] = label_value
            label_value += 1

        class_id = label_map[label]
        rle = v["rle"]

        # Decodificar RLE en binario
        try:
            if isinstance(rle, dict):
                decoded_mask = decode_rle(rle["counts"], rle["size"])
            else:
                decoded_mask = decode_rle(rle, (height, width))
        except Exception as e:
            print(f"⚠️ Bad RLE in {json_file}: {e}")
            continue

        decoded_mask = np.asarray(decoded_mask)

        # Formatos posibles: (H*W), (H*W*4), (H,W), (H,W,4)
        if decoded_mask.ndim == 1 and decoded_mask.size == height * width:
            binary_mask = decoded_mask.reshape((height, width))
        elif decoded_mask.ndim == 1 and decoded_mask.size == height * width * 4:
            rgba = decoded_mask.reshape((height, width, 4))
            binary_mask = rgba[:, :, 3]  # alpha
        elif decoded_mask.ndim == 2:
            binary_mask = decoded_mask
        elif decoded_mask.ndim == 3:
            if decoded_mask.shape[2] >= 4:
                binary_mask = decoded_mask[:, :, 3]
            else:
                binary_mask = np.any(decoded_mask > 0, axis=2).astype(np.uint8) * 255
        else:
            print(f"⚠️ Unexpected decoded shape in {json_file}: {decoded_mask.shape}")
            continue

        # ✅ Merge rule: brush overwrites polygon
        mask[binary_mask > 0] = class_id

    # === Crear y guardar máscara coloreada usando colores fijos ===
    mask_rgb = np.zeros((height, width, 3), dtype=np.uint8)

    for label_name, label_idx in label_map.items():
        color = np.array(class_colors.get(label_name, (255, 255, 255)), dtype=np.uint8)
        mask_rgb[mask == label_idx] = color

    base_name = os.path.splitext(json_file)[0]
    color_output_path = os.path.join(output_color_mask_folder, f"{base_name}.png")
    Image.fromarray(mask_rgb).save(color_output_path)
    print(f"✅ Máscara coloreada guardada: {color_output_path}")

print("\n✅ Todos los JSON (polygons + brush) han sido convertidos a máscaras coloreadas PNG.")


RLE params: 1228800 values 8 word_size [3, 4, 8, 16] rle_sizes
✅ Máscara coloreada guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_sam2_tir/task_0001.png
RLE params: 1228800 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1228800 values 8 word_size [3, 4, 8, 16] rle_sizes
✅ Máscara coloreada guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_sam2_tir/task_0002.png
RLE params: 1228800 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1228800 values 8 word_size [3, 4, 8, 16] rle_sizes
✅ Máscara coloreada guardada: /workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/mfnet/json_to_color_sam2_tir/task_0003.png
RLE params: 1228800 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1228800 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1228800 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1228800 values 8 word_size [3, 